In [0]:
spark

In [0]:
dbutils.fs.mount(source='wasbs://landing@week17storage.blob.core.windows.net',mount_point='/mnt/week11assignment/landing',
extra_configs={'fs.azure.account.key.week17storage.blob.core.windows.net':'DwezHFxwgpDba4/fh4P6oR3TlTZYUoJTggu4PIGhSt5AF23lMRBi9yfG6tfOtR666VhUlFexopS2+AStge7HnQ=='})

True

In [0]:
dbutils.fs.mount(source='wasbs://reporting@week17storage.blob.core.windows.net',mount_point='/mnt/week11assignment/reporting',
extra_configs={'fs.azure.account.key.week17storage.blob.core.windows.net':'DwezHFxwgpDba4/fh4P6oR3TlTZYUoJTggu4PIGhSt5AF23lMRBi9yfG6tfOtR666VhUlFexopS2+AStge7HnQ=='})

True

In [0]:
dbutils.fs.mount(source='wasbs://staging@week17storage.blob.core.windows.net',mount_point='/mnt/week11assignment/staging',
extra_configs={'fs.azure.account.key.week17storage.blob.core.windows.net':'DwezHFxwgpDba4/fh4P6oR3TlTZYUoJTggu4PIGhSt5AF23lMRBi9yfG6tfOtR666VhUlFexopS2+AStge7HnQ=='})

True

In [0]:
order_schema='order_id String,order_date date,order_customer_id String,order_status String'

In [0]:
print(order_schema)

order_id String,order_date date,order_customer_id String,order_status String


In [0]:
orders_df=spark.read \
    .format("csv") \
    .schema(order_schema) \
    .load("dbfs:/mnt/week11assignment/landing/orders_data.csv")

In [0]:
orders_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- order_customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)



In [0]:
orders_df.show(5)

+--------+----------+-----------------+---------------+
|order_id|order_date|order_customer_id|   order_status|
+--------+----------+-----------------+---------------+
|       1|2013-07-25|            11599|         CLOSED|
|       2|2013-07-25|              256|PENDING_PAYMENT|
|       3|2013-07-25|            12111|       COMPLETE|
|       4|2013-07-25|             8827|         CLOSED|
|       5|2013-07-25|            11318|       COMPLETE|
+--------+----------+-----------------+---------------+
only showing top 5 rows



In [0]:
customer_schema='customer_id string,customer_fname string,customer_lname string,customer_email string,customer_password string,customer_street string,customer_city string,customer_state string,customer_zipcode string'

In [0]:
print(customer_schema)

customer_id string,customer_fname string,customer_lname string,customer_email string,customer_password string,customer_street string,customer_city string,customer_state string,customer_zipcode string


In [0]:
customer_df=spark.read \
    .format("csv") \
    .schema(customer_schema) \
    .load('dbfs:/mnt/week11assignment/landing/customers_data.csv')

In [0]:
customer_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_fname: string (nullable = true)
 |-- customer_lname: string (nullable = true)
 |-- customer_email: string (nullable = true)
 |-- customer_password: string (nullable = true)
 |-- customer_street: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zipcode: string (nullable = true)



In [0]:
from pyspark.sql.functions import *

In [0]:
orders_new_df=orders_df.withColumn("order_year",year("order_date")).withColumn("order_month",month("order_date"))

In [0]:
orders_new_df.show(5)

+--------+----------+-----------------+---------------+----------+-----------+
|order_id|order_date|order_customer_id|   order_status|order_year|order_month|
+--------+----------+-----------------+---------------+----------+-----------+
|       1|2013-07-25|            11599|         CLOSED|      2013|          7|
|       2|2013-07-25|              256|PENDING_PAYMENT|      2013|          7|
|       3|2013-07-25|            12111|       COMPLETE|      2013|          7|
|       4|2013-07-25|             8827|         CLOSED|      2013|          7|
|       5|2013-07-25|            11318|       COMPLETE|      2013|          7|
+--------+----------+-----------------+---------------+----------+-----------+
only showing top 5 rows



In [0]:
orders_new_df.write \
. format("parquet") \
. mode("overwrite") \
.partitionBy("order_year","order_status") \
.save("/mnt/week11assignment/staging/orders")

In [0]:
customer_df.write \
.format("parquet") \
.mode("overwrite") \
.partitionBy("customer_state") \
.save("/mnt/week11assignment/staging/customers")

In [0]:
customer_staging_df=spark.read \
    .format("parquet") \
    .schema(customer_schema) \
    .load('dbfs:/mnt/week11assignment/staging/customers')

In [0]:
orders_staging_df=spark.read \
    .format("parquet") \
    .option("header",True) \
    .load("dbfs:/mnt/week11assignment/staging/orders")

In [0]:
customer_staging_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_fname: string (nullable = true)
 |-- customer_lname: string (nullable = true)
 |-- customer_email: string (nullable = true)
 |-- customer_password: string (nullable = true)
 |-- customer_street: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_zipcode: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [0]:
orders_staging_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- order_customer_id: string (nullable = true)
 |-- order_month: integer (nullable = true)
 |-- order_year: integer (nullable = true)
 |-- order_status: string (nullable = true)



In [0]:
joined_df=orders_staging_df.join(broadcast(customer_staging_df),expr("order_customer_id=customer_id"))

In [0]:
joined_df.show(5)

+--------+----------+-----------------+-----------+----------+------------+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+----------------+--------------+
|order_id|order_date|order_customer_id|order_month|order_year|order_status|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_zipcode|customer_state|
+--------+----------+-----------------+-----------+----------+------------+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+----------------+--------------+
|   68883|2014-07-23|             5533|          7|      2014|    COMPLETE|       5533|          Mary|      Harrison|     XXXXXXXXX|        XXXXXXXXX|852 Rocky Brook M...|       Caguas|           00725|            PR|
|   68880|2014-07-13|             1117|          7|      2014|    COMPLETE|       1117|        Arthur|        Maddox|     XXXXXX

In [0]:
report_df=joined_df.select("order_id","order_date","order_customer_id","order_month","order_year","order_status","customer_fname","customer_lname","customer_city","customer_zipcode","customer_state")

In [0]:
report_df.write \
    .format("parquet") \
    .mode("overwrite") \
    .partitionBy("order_year","customer_state","customer_city","order_status") \
    .save("/mnt/week11assignment/reporting")

In [0]:
report_df.createOrReplaceTempView("order_customer")

In [0]:
report_new_df=spark.read \
    .format("parquet") \
    .option("header","true") \
    .load("/mnt/week11assignment/reporting")